# Downloading PlanetScope Scenes using Planet Orders API
---

**Objectives:**

By the end of this exercise, you should be able to:
* to access Planet Orders API
* to search for PlanetScope Scenes given a set of rules (e.g., using a area of interest to filter out images)
* to download multiple PlanetScope Scenes using Planet Orders API
---

We will continue to use Lake Raleigh as our area of interest (AOI). Planet has multiple [APIs](https://developers.planet.com/docs/apis/) that serve different purposes. In this exercise, we will use the [Orders API](https://developers.planet.com/apis/orders/). This API is used to access analysis ready data (e.g., surface reflectance imagery) and have the images delivered directly to our local or cloud storage. 

The Orders API makes it easier to create pipelines to continuously downloading imagery for processing and analysis. For instance, you can manually download imagery using [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or Planet [QGIS](https://developers.planet.com/docs/integrations/qgis/) or [ArcGIS](https://developers.planet.com/docs/integrations/arcgis/); nonetheless, this quickly becomes a burden if you have to download thousands of images for multiple AOIs!   

#### Getting Started with Planet APIs

To use any Planet API, you'll need an API key. API keys are available to all registered users with active Planet accounts. Once you signed up for the [Education and Research Program](https://www.planet.com/markets/education-and-research/) account or the CGA's license, you will need to get your API key. To do so, log in to your account at [planet.com/account](https://www.planet.com/account/). 

<p align="center">
    <img src='imgs/planet-api-key.png' width='1200' /> 
</p>

In [41]:
import os
import time
import json
import shapely
import geojson
import requests
import numpy as np
import pandas as pd
from pathlib import Path

from shapely import geometry as sgeom
from requests.auth import HTTPBasicAuth

#### Authentication

You will need your Planet API Key here. The easiest way to authenticate is to simply copy and past the API key into the Jupyter Notebook—we will use this method to make it simpler. Nonetheless, this is not recommended, as this document maybe public (i.e., in your GitHub account) or you may share it with others. Therefore, for future reference, it is recommended that you store your API key as an environment variable, read more about [here](https://www.nylas.com/blog/making-use-of-environment-variables-in-python/). 

In [3]:
PL_API_KEY = 'copy your API key here' 

# URL to access the Orders API
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

To communicate with the API we will use the python package **requests**. First, we will make sure that the authentication and communication are working as expected. We expect to get a response code of **200** from this API call. To troubleshoot other response codes, please check [here](https://developers.planet.com/docs/orders/reference/#operation/listOrders). 

In [4]:
auth = HTTPBasicAuth(PL_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

Now that the communication is working fine, we will build up a request to search for images. We will load our geojson (AOI for Lake Raleigh), and define some objects, for example, cloud cover, dates, etc.  

### Loading the coordinates (AOI) from the lake-raleigh.geojson

In [5]:
with open('aoi/lake-raleigh.geojson') as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates'][0]
aoi_geom = {"type": "Polygon","coordinates": coords}
aoi_geom

{'type': 'Polygon',
 'coordinates': [[[-78.687957, 35.769435],
   [-78.675624, 35.769435],
   [-78.675691, 35.76139],
   [-78.688092, 35.761281],
   [-78.687957, 35.769435]]]}

### Defining the general objects for imagery search

First, we need to define a time frame for our search at Planet's catalog, this includes a starting and an ending date. Then, we will define a cloud cover threshold (from 0 to 1) with a less than or equal (lte) rule, and the type of imagery that we want. Planet has several item [types](https://developers.planet.com/docs/apis/data/items-assets/), we will be using [PSScenes](https://developers.planet.com/docs/data/psscene/), and we will download the following asset types: 'ortho_analytic_4b' and 'ortho_udm2'. See all available asset types [here](https://developers.planet.com/docs/data/psscene/).

In [6]:
st_date, end_date = '2021-10-01', '2021-11-15'
cloud_cover = 0.05 # 5%
item_type = 'PSScene'
asset_one, asset_two = 'ortho_analytic_4b', 'ortho_udm2'

After defining the objects, we need to pass them along as dictionaries to create the request body that we will send to the API. We have a total of four filters—geometry, date, cloud and asset type—in form of dictionaries. More examples can be found [here](https://developers.planet.com/docs/apis/data/searches-filtering/).

In [7]:
geometry_filter = { "type": "GeometryFilter",
                    "field_name": "geometry",
                    "config": aoi_geom}

date_filter = {"type": "DateRangeFilter",
               "field_name": "acquired",
               "config": {"gte": "{}T00:00:00.000Z".format(st_date),"lte": "{}T23:59:59.999Z".format(end_date)}}

cloud_filter = {"type": "RangeFilter",
                "field_name": "cloud_cover",
                "config": {"lte": cloud_cover}}
                
asset_type = {"type": "AndFilter",
              "config": [{"type": "AssetFilter","config": [asset_one]},{"type": "AssetFilter","config": [asset_two]}]}

combined_filter = {"type": "AndFilter",
                   "config": [geometry_filter,date_filter,cloud_filter,asset_type]}

search_request = {"item_types": [item_type],
                        "filter": combined_filter}
search_request

{'item_types': ['PSScene'],
 'filter': {'type': 'AndFilter',
  'config': [{'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {'type': 'Polygon',
     'coordinates': [[[-78.687957, 35.769435],
       [-78.675624, 35.769435],
       [-78.675691, 35.76139],
       [-78.688092, 35.761281],
       [-78.687957, 35.769435]]]}},
   {'type': 'DateRangeFilter',
    'field_name': 'acquired',
    'config': {'gte': '2021-10-01T00:00:00.000Z',
     'lte': '2021-11-15T23:59:59.999Z'}},
   {'type': 'RangeFilter',
    'field_name': 'cloud_cover',
    'config': {'lte': 0.05}},
   {'type': 'AndFilter',
    'config': [{'type': 'AssetFilter', 'config': ['ortho_analytic_4b']},
     {'type': 'AssetFilter', 'config': ['ortho_udm2']}]}]}}

### Posting the search request
Now that we created the search request with the imagery filtering rules that we want, we will build a **post** request to interact with the API. Again, if sucessfull, we should expect **200** as the post response.

In [8]:
url_quick_search = "https://api.planet.com/data/v1/quick-search" # url to search for imagery

ses = requests.Session() 
res = ses.post(url_quick_search, auth=(PL_API_KEY,''), json=search_request)

if not res.status_code == 200:
    print(res.text)
else:
    image_ids = [feature["id"] for feature in res.json()["features"]]
    if ((len(image_ids))) == 0:
        print(("No suitable images found were found. Double check filters, including asset_types."))
    else:
        print("Found {} images matching our filters.".format(len(image_ids)))

# get response as json, this object contains the metadata for all images in our search
result = res.json()
metadata = pd.DataFrame([{**{'image_id':img['id']},**img['properties']} for img in result['features']])
metadata.head() 

Found 42 images matching our filters.


,image_id,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,cloud_percent,ground_control,gsd,heavy_haze_percent,...,satellite_id,shadow_percent,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent
0,20211114_160039_40_2402,2021-11-14T16:00:39.405833Z,0.00,99,100,0.0,0,True,4.0,0,...,2402,0,0,5098663,162.6,33.9,2021-11-16T17:32:46Z,4.1,73,100
1,20211113_153332_100a,2021-11-13T15:33:32.984977Z,0.07,96,100,0.0,0,True,3.9,0,...,100a,0,0,5092879,156.2,32.6,2021-11-14T04:42:59Z,5.0,83,100
2,20211113_153333_100a,2021-11-13T15:33:33.998426Z,0.07,99,100,0.0,0,True,3.9,0,...,100a,0,0,5092879,156.1,32.7,2021-11-14T04:42:57Z,5.0,82,100
3,20211113_150842_82_245a,2021-11-13T15:08:42.825984Z,0.00,99,100,0.0,0,True,4.1,0,...,245a,0,0,5092473,149.0,30.0,2021-11-14T05:05:13Z,5.0,72,100
4,20211113_150840_52_245a,2021-11-13T15:08:40.526023Z,0.00,99,100,0.0,0,True,4.1,0,...,245a,0,0,5092473,149.1,29.9,2021-11-14T05:05:14Z,5.0,76,100


### Refining our search

When using Planet Explorer, we are able to set an AOI coverage (%) filter, which tells us how much of the AOI is covered by the image. This is very important because it enables us to filter out images that cover only a small part of our AOI. For instance, see the example below, in which only the blue shaded part of the AOI is covered by the image. The Orders API does not have a built in filter, therefore, we will create our own method to filter out those images.

We will use the images' metadata from our search to calculate the percentage of overlap between the image extent and the AOI extent. 

<p align="center">
    <img src='imgs/planet-aoi-coverage.png' width='1200' /> 
</p>

In [9]:
def shapelyAOI(geom):
    '''to build AOI using shape, from shapely, structure.'''
    aoi = {u'geometry': {u'type': u'Polygon', u'coordinates': geom['coordinates']}}
    aoi_shape = sgeom.shape(aoi['geometry'])
    
    return aoi_shape

# transform aoi_geom to shapely shape
aoi_shape = shapelyAOI(aoi_geom)

# get the shapely shape for all images from our response (i.e., result)
images_shapes = [shapelyAOI(geom['geometry']) for geom in result['features']]

_cov = []
for index,image_shape in enumerate(images_shapes):
    coverage = np.round((aoi_shape.intersection(image_shape).area/aoi_shape.area)*100,2)
    _cov.append(coverage)

metadata['coverage'] = _cov
metadata.head()

,image_id,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,cloud_percent,ground_control,gsd,heavy_haze_percent,...,shadow_percent,snow_ice_percent,strip_id,sun_azimuth,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,coverage
0,20211114_160039_40_2402,2021-11-14T16:00:39.405833Z,0.00,99,100,0.0,0,True,4.0,0,...,0,0,5098663,162.6,33.9,2021-11-16T17:32:46Z,4.1,73,100,100.00
1,20211113_153332_100a,2021-11-13T15:33:32.984977Z,0.07,96,100,0.0,0,True,3.9,0,...,0,0,5092879,156.2,32.6,2021-11-14T04:42:59Z,5.0,83,100,82.17
2,20211113_153333_100a,2021-11-13T15:33:33.998426Z,0.07,99,100,0.0,0,True,3.9,0,...,0,0,5092879,156.1,32.7,2021-11-14T04:42:57Z,5.0,82,100,100.00
3,20211113_150842_82_245a,2021-11-13T15:08:42.825984Z,0.00,99,100,0.0,0,True,4.1,0,...,0,0,5092473,149.0,30.0,2021-11-14T05:05:13Z,5.0,72,100,100.00
4,20211113_150840_52_245a,2021-11-13T15:08:40.526023Z,0.00,99,100,0.0,0,True,4.1,0,...,0,0,5092473,149.1,29.9,2021-11-14T05:05:14Z,5.0,76,100,100.00


In [35]:
coverage_thresh = 90
images_subset = metadata[metadata.coverage>=coverage_thresh]
print(f'After filtering out images that did not cover {coverage_thresh}% or more of our AOI, were are left with {len(images_subset)} images.')

After filtering out images that did not 90% or more of our AOI, were are left with 32 images.


We also have images that were collected on the same day, within minutes or hours apart. After we applied our AOI coverage filter, all images that are left cover >= 90% of our AOI. In this case, we could use another set of rules, based on the images' metadata, to decide which images to chose from those that were collected on the same day. To make it simpler, we will chose the image that was collected first, using the 'acquired' column from images_subset. 

In [36]:
dates = [d[0:10] for d in images_subset.acquired.tolist()]
images_subset = images_subset.assign(dates=dates) # create new column called 'dates' using assign

# drop duplicates using column 'dates'
images_subset = images_subset.drop_duplicates(subset='dates',keep='first')
print(f'After filtering out images that were collected on the same day, were are left with {len(images_subset)} images.')

After filtering out images that were collected on the same day, were are left with 21 images.


### Place imagery order, activate items and download imagery

To download the images, we need their image id, which is available within image metadata. 

In [38]:
len(images_subset.image_id.tolist())

21

In [43]:
# general objects
image_ids = images_subset.image_id.tolist() # images that will be downloaded
url_orders = 'https://api.planet.com/compute/ops/orders/v2' # POST
headers = {'content-type': 'application/json'}
order_name = 'lake_raleigh'
save_path = os.path.join(os.getcwd(),'results') # you can change if you would like to save in another folder

### Creating and posting the payload for imagery download

In [40]:
# create post payload
payload = {
    "name": order_name,
    "order_type": "partial",
    "products": [{
        "item_ids": image_ids,
        "item_type": 'PSScene',
        "product_bundle": 'analytic_sr_udm2'
    }],
    "tools": [
        {
            "harmonize": {
                "target_sensor": "Sentinel-2"
            }},
        {
            "clip": {
                "aoi": aoi_geom # Lake Raleigh extent
            }
        }
    ],
    "delivery": {},
    "notifications": {
        "email": False
    }
}

response = requests.post(url_orders, data=json.dumps(payload), auth=(PL_API_KEY,''), headers=headers)

if response.status_code == 202:
    print("Order succesfully placed with order id {}".format(response.json()["id"]))
    order_id = response.json()["id"]
    order_url = url_orders + "/" + order_id
    print(f'Order url: \n {order_url}')
else:
    print("Order failed with error {}".format(response.json()))

Order succesfully placed with order id b5c350c4-c092-48c8-b20c-49e5ab4175ca
Order url: 
 https://api.planet.com/compute/ops/orders/v2/b5c350c4-c092-48c8-b20c-49e5ab4175ca


### Checking the order status, polling for success, and downloading the images.

In [44]:
def poll_for_success(order_url,num_loops=100):
    print("Polling order..")
    count = 0
    while (count < num_loops):
        count += 1
        r = requests.get(order_url, auth=(PL_API_KEY,''))
        response = r.json()
        state = response["state"]
        print("Current state: {}".format(state))
        end_states = ["success", "failed", "partial"]
        if state in end_states:
            results = response["_links"]["results"]
            results_name = [r["name"] for r in results]
            results_urls = [r["location"] for r in results]
            return [results_name, results_urls]
        time.sleep(60)

location_url_list = poll_for_success(order_url)

Polling order..
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: running
Current state: success


### Downloading the images once the order is ready (current state = *'success'*).  

We will download the images and save them at the 'save_path' directory that we defined above. The download_results() will the create the 'save_path' directory if does not exists. 

In [45]:
def download_results(results_urls, results_name, overwrite=False):
    print("{} items to download".format(len(results_urls)))
    for url, name in zip(results_urls, results_name):
        path = Path(save_path + f'/images/{order_name}/{name}')
        if overwrite or not path.exists():
            print("Downloading : {}".format(os.path.basename(path)))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, "wb").write(r.content)
        else:
            print("{} :Already exists, skipping".format(os.path.basename(path)))

if len(location_url_list) > 0:
    results_names = location_url_list[0]
    results_urls = location_url_list[1]
    download_results(results_urls, results_names)

85 items to download
